In [2]:
# try to use ansamble 

from processing_module import * 
import matplotlib.pyplot as plt

offset = 0
file_columns = ['id','rle_mask']

n_NN = 6
models = [{} for x in range(n_NN)]
models[0]['filename'] = 'ensamble/0_217_229'
models[0]['L'] = 4+1 #L
models[0]['add_xy'] = True

models[1]['filename'] = 'ensamble/1_207_135'
models[1]['L'] = 6+1 #L
models[1]['add_xy'] = True

models[2]['filename'] = 'ensamble/2_199_60'
models[2]['L'] = 6+1 #L
models[2]['add_xy'] = False

models[3]['filename'] = 'ensamble/3_227_24'
models[3]['L'] = 4+1 #L
models[3]['add_xy'] = False

models[4]['filename'] = 'ensamble/4_241_170'
models[4]['L'] = 6+1 #L
models[4]['add_xy'] = False

models[5]['filename'] = 'ensamble/5_242_80'
models[5]['L'] = 6+1 #L
models[5]['add_xy'] = False

print('train {} models'.format(n_NN))
#print(get_columns_list(L))

#parameters = [None for x in range(n_NN)]
#print(parameters)
for i, model in enumerate(models): 
    L = model['L']
    models[i]['parameters'] = load_parameters(model, L)
    if (L-1 == 4):
        f_prop = forward_propagation4
    if (L-1 == 5):
        f_prop = forward_propagation5
    if (L-1 == 6):
        f_prop = forward_propagation6    
    models[i]['f_prop'] = f_prop
    print('name: {}, L: {}'.format(model['filename'],model['L']))
        

test_file_obj = {}
test_file_obj['test_images_path'] = '../images'
test_file_obj['test_image_files'] = get_testfiles_list(test_file_obj['test_images_path'])

#test_file_obj['test_images_path'] = '../train/images'
#test_file_obj['test_image_files'] = get_testfiles_list(test_file_obj['test_images_path'])


test_file_obj['deep_filename'] = '../depths.csv'
test_files = test_file_obj['test_image_files']
test_file_obj['result_file'] = 'results/res_10-19'

print('files to process: {}'.format(len(test_files)))


train 6 models
name: ensamble/0_217_229, L: 5
name: ensamble/1_207_135, L: 7
name: ensamble/2_199_60, L: 7
name: ensamble/3_227_24, L: 5
name: ensamble/4_241_170, L: 7
name: ensamble/5_242_80, L: 7
files to process: 18000


In [3]:
start_time = time.time()
file_frame = convert_resfile_to_frame(test_file_obj)
print('processing frame: {}'.format(time.time() - start_time))

processing frame: 64.81027674674988


In [4]:
if isfile(test_file_obj['result_file']+'.csv'):
    res_df = pd.read_csv(test_file_obj['result_file']+'.csv', names = file_columns)
    offset = len(res_df.index)-1
    print('continue file: {} from pos# {}:'.format(test_file_obj['result_file']+'.csv', offset))
else:
    res_df = pd.DataFrame(columns = ['id','rle_mask']).set_index('id')
    res_df.to_csv(test_file_obj['result_file']+'.csv', mode='w', header=True)

start_time = time.time()
beg_t = time.time()
iter_t = time.time()
for i,f in enumerate(test_files):

    #check = [None for x in range(1)]

    
    img_array = file_frame.loc[f]['2D-img']
    deep = file_frame.loc[f]['z1']
    fname = file_frame.loc[f]['fname']


    frame = processing_single_testfile(fname, img_array, deep)    
    
    
    res_df = pd.DataFrame(columns = file_columns)
    Y = [None for x in range(n_NN)]
    for itm, model in enumerate(models):
        if model['add_xy']:
            feature_X = 'extended_scaled_surround_xy'
        else:
            feature_X = 'extended_scaled_surround'

        X_data_arr = frame[feature_X]
        X  = np.array([x for x in X_data_arr]).T
        Y[itm], _ = model['f_prop'](X, model['parameters'])
        
    Yavg = (Y[0] + Y[1] + Y[2] + Y[3] + Y[4] + Y[5])/6
    frame['prediction'] = (Yavg.T >0.65)*1  

    #frame['prediction'] = [((f_prop(x.reshape(-1,1), parameters)[0])[0][0] >0.5)*1 for x in frame['extended_scaled_surround']]
    test_predict = frame.sort_values(['px_id'])
    
    pic_res = np.array(test_predict['prediction']).reshape(101,101)
    
    item = {}
    item['id'] = f.split('.')[0]
    item['rle_mask'] = convert_mask(pic_res)
    res_df = res_df.append([item], sort = False).set_index('id')
    res_df.to_csv(test_file_obj['result_file']+'.csv', mode='a', header=False)
    
    #check[0] = time.time() - beg_t
    if i%250 == 0:
        point_time = time.strftime('%d.%m.%Y %H:%M:%S', time.localtime())
        print('i: {}, i+offset: {}, iter time: {:}, total time: {:}, ch-p: {:}'.format(i,i+offset,(time.time()- iter_t)/60,(time.time()- start_time)/60, point_time))
        iter_t = time.time()
    #print(check)
    
#frame.head()
print('thats it')

i: 0, i+offset: 0, iter time: 0.02329500118891398, total time: 0.023295533657073975, ch-p: 19.10.2018 11:20:05
i: 250, i+offset: 250, iter time: 3.0322827339172362, total time: 3.0555840810139974, ch-p: 19.10.2018 11:23:07
i: 500, i+offset: 500, iter time: 2.8554275194803873, total time: 5.911013201872508, ch-p: 19.10.2018 11:25:59
i: 750, i+offset: 750, iter time: 2.9094202478726703, total time: 8.820435130596161, ch-p: 19.10.2018 11:28:53
i: 1000, i+offset: 1000, iter time: 2.955975317955017, total time: 11.776412431399027, ch-p: 19.10.2018 11:31:51
i: 1250, i+offset: 1250, iter time: 8.978206034501394, total time: 20.75462044874827, ch-p: 19.10.2018 11:40:49
i: 1500, i+offset: 1500, iter time: 3.0651352326075236, total time: 23.819757282733917, ch-p: 19.10.2018 11:43:53
i: 1750, i+offset: 1750, iter time: 3.1804588794708253, total time: 27.00021804968516, ch-p: 19.10.2018 11:47:04
i: 2000, i+offset: 2000, iter time: 3.0896976351737977, total time: 30.08992042938868, ch-p: 19.10.2018

In [ ]:
def visualisate(f):
    check = [None for x in range(6)]

    msk_array = Image.open('../train/masks' + '/'+ f).convert(mode='L')

    st_t = time.time()
    beg_t = time.time()

    img_array = file_frame.loc[f]['2D-img']  #step 1
    deep = file_frame.loc[f]['z1'] # step 1
    fname = file_frame.loc[f]['fname']  #step 1
    check[1] = time.time() - st_t

    st_t = time.time()
    frame = processing_single_testfile(fname, img_array, deep)  # step 2
    check[2] = time.time() - st_t


    st_t = time.time()
    #test_predict = frame.apply(predict,args=(f_prop, parameters), axis = 1) #step 3
    #test_predict['prediction'] = [type(f_prop(x.reshape(-1,1), parameters)[0]) for x in frame['extended_scaled_surround']]
    #frame['prediction'] = [((f_prop(x.reshape(-1,1), parameters)[0])[0][0] >0.5)*1 for x in frame['extended_scaled_surround']]

    X_data_arr = frame['extended_scaled_surround']
    X  = np.array([x for x in X_data_arr]).T
    Y, _ = f_prop(X, parameters)
    frame['prediction'] = (Y.T >0.5)*1
    check[3] = time.time() - st_t

    st_t = time.time()
    test_predict = frame.sort_values(['px_id'])  #step 4
    check[4] = time.time() - st_t

    st_t = time.time()
    pic_res = np.array(test_predict['prediction']).reshape(101,101) #step 5
    check[5] = time.time() - st_t
    check[0] = time.time() - beg_t

    fig, ax =  plt.subplots(1, 3, figsize=(9, 5), sharey=False, sharex = False)

    plt.title(str(id) + ' : ' + f)
    ax[0].imshow(img_array, cmap='gray')  
    ax[1].imshow(msk_array, cmap='gray')
    ax[2].imshow(pic_res, cmap='gray')

    plt.show()
    print(check)
    return frame
    
tt = visualisate(file_frame.index[8])

In [11]:
## clearing result file

In [27]:
frame = pd.read_csv(test_file_obj['result_file']+ '_3.csv').set_index('id')

In [28]:
len(frame['rle_mask'][0].split(' '))

16

In [29]:
frame['clr'] = [len(str(x).split(' '))<=8 for x in frame['rle_mask']]
#ind = frame['clr'] == True
#print(ind)
frame.loc[frame['clr'] == True, 'rle_mask'] = np.nan
frame1 = frame[['rle_mask']]

In [30]:
frame.head(50)

,rle_mask,clr
id,,
dc066ad24e,1278 2 1380 3 1483 3 1589 2 1691 6 1794 6 1898...,False
9ddc91e5bb,442 2 449 18 478 16 495 3 540 6 547 26 574 2 5...,False
ef441205b5,8600 2 8603 3 8700 12 8800 15 8901 22 9005 5 9...,False
4a723a302f,4655 6 4749 2 5183 3 5199 3 5260 4 5286 8 5298...,False
16545fae06,NaN,True
b258011964,NaN,True
e50303e172,NaN,True
ccc645a996,NaN,True
7efd271bba,NaN,True


In [31]:
frame1.to_csv(test_file_obj['result_file']+'_4.csv', mode='w', header=True)

In [19]:
def erase_ones(row):
    lst = str(row['rle_mask']).split(' ')
    lst1 = list(zip(lst[0::2],lst[1::2]))
    
    if len(lst1) > 0:
        erased = [x[0]+ ' ' + x[1] for x in lst1 if int(x[1])>1]
        row['rle_mask'] = str(erased).replace('[','').replace(']','').replace(',','').replace("'",'')
    return row

In [88]:
erase_ones('1198 3 1277 3 1302 2 1379 4 1482 4 1494 3')

[('1198', '3'), ('1277', '3'), ('1302', '2'), ('1379', '4'), ('1482', '4'), ('1494', '3')]


'1198 3 1277 3 1302 2 1379 4 1482 4 1494 3'

In [20]:
frame3 = pd.read_csv(test_file_obj['result_file']+ '_3.csv').set_index('id')
#frame4 = frame3.apply(erase_ones)
frame3.head()

,rle_mask
id,
dc066ad24e,1199 1 1278 2 1380 3 1483 3 1589 2 1691 6 1794...
9ddc91e5bb,356 1 442 2 449 18 478 16 495 3 540 6 547 26 5...
ef441205b5,8600 2 8603 3 8700 12 8800 15 8897 1 8901 22 8...
4a723a302f,935 1 4655 6 4749 2 4759 1 5183 3 5199 3 5260 ...
16545fae06,NaN


In [21]:
frame4 = frame3.apply(erase_ones, axis =1)

In [22]:
frame4.head()

,rle_mask
id,
dc066ad24e,1278 2 1380 3 1483 3 1589 2 1691 6 1794 6 1898...
9ddc91e5bb,442 2 449 18 478 16 495 3 540 6 547 26 574 2 5...
ef441205b5,8600 2 8603 3 8700 12 8800 15 8901 22 9005 5 9...
4a723a302f,4655 6 4749 2 5183 3 5199 3 5260 4 5286 8 5298...
16545fae06,NaN


In [23]:
frame4.to_csv(test_file_obj['result_file']+'_3.csv', mode='w', header=True)